In [3]:
#from f3dasm import ExperimentData
import numpy as np
from pathlib import Path
import pandas as pd
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures # For Polynomial basis functions
from sklearn.pipeline import make_pipeline # to link different objects
from matplotlib import cm # to change colors of surface plots
import matplotlib.pyplot as plt # import plotting tools to create figures
from sklearn.gaussian_process import GaussianProcessRegressor,GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, Matern, ExpSineSquared, ConstantKernel, WhiteKernel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import r2_score, mean_squared_error
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score


In [9]:
df_in_7d = pd.read_csv('data/supercompressible_7d_input.csv')
df_out_7d = pd.read_csv('data/supercompressible_7d_output.csv')

df_in_3d = pd.read_csv('data/supercompressible_3d_input.csv')
df_out_3d = pd.read_csv('data/supercompressible_3d_output.csv')

In [3]:
# define a function to do the preprocessing
# Input: input_raw_data, output_raw_data, problem_class
# input_raw_data: pandas dataframe for input
# output_raw_data: pandas dataframe for output
# problem_class: (booler) to describe this preprocess is for classification problem or not
#                this will lead to different scaler for Y data.
#
# Output: X_train_scale, X_test_scale, X_scale, Y_train_scale, Y_test_scale, Y_scale, scaler_x, scaler_y
# scaler_x: (scaler) used to do inverse_transfer after prediction
# scaler_y: (scaler) used to do inverse_transfer after prediction

def preprocess_3d(input_raw_data,output_raw_data,problem_class):
    if problem_class == False:
        raw_data = pd.concat([input_raw_data, output_raw_data], axis=1)

        # look at the # of missing points in the first ten columns
        raw_data_drop_nan = raw_data.dropna()

        X_3d = raw_data_drop_nan.loc[:, ['ratio_d','ratio_pitch','ratio_top_diameter'] ].values
        Y_3d = raw_data_drop_nan.loc[:, ['sigma_crit','energy'] ].values
        
        testset_ratio = 0.25
        SEED = 123
        X_train, X_test, Y_train, Y_test = train_test_split(X_3d,
                                            Y_3d, test_size=testset_ratio,
                                            random_state=SEED)

        scaler_x = StandardScaler()
        scaler_x.fit(X_train)
        X_train_scale=scaler_x.transform(X_train)
        X_test_scale=scaler_x.transform(X_test)
        X_scale=scaler_x.transform(X_3d)
        
        scaler_y = StandardScaler()
        scaler_y.fit(Y_train)
        Y_train_scale = scaler_y.transform(Y_train)
        Y_test_scale = scaler_y.transform(Y_test)
        Y_scale = scaler_y.transform(Y_3d)
    else:
        X_3d = input_raw_data.loc[:, ['ratio_d','ratio_pitch','ratio_top_diameter'] ].values
        Y_3d = output_raw_data.loc[:, 'coilable'].values
        
        testset_ratio = 0.25
        SEED = 123
        X_train, X_test, Y_train, Y_test = train_test_split(X_3d,
                                            Y_3d, test_size=testset_ratio,
                                            random_state=SEED)

        scaler_x = StandardScaler()
        scaler_x.fit(X_train)
        X_train_scale=scaler_x.transform(X_train)
        X_test_scale=scaler_x.transform(X_test)
        X_scale=scaler_x.transform(X_3d)

        scaler_y = FunctionTransformer() # FunctionTransformer without input will give a Identity scaler
        Y_train_scale = scaler_y.transform(Y_train)
        Y_test_scale = scaler_y.transform(Y_test)
        Y_scale = scaler_y.transform(Y_3d)

    return X_train_scale, X_test_scale, X_scale, Y_train_scale, Y_test_scale, Y_scale, scaler_x, scaler_y

In [4]:
X_train_scale_class, X_test_scale_class, X_scale_class, Y_train_scale_class, Y_test_scale_class, Y_scale_class, scaler_x_class, scaler_y_class = preprocess_3d(df_in_3d,df_out_3d,True)
X_train_scale_regression, X_test_scale_regression, X_scale_regression, Y_train_scale_regression, Y_test_scale_regression, Y_scale_regression, scaler_x_regression, scaler_y_regression = preprocess_3d(df_in_3d,df_out_3d,False)

In [11]:
# Define function to display classification when fixing "ratio_top_diameter"
# Input: X_data,Y_data,scaler_x, scaler_y,model,sample_index,grid_num=20
# X_data, Y_data, scaler_x: scaled data set for classification
# model: any model for predict classification problem
# sample_index: index of "ratio_top_diameter" pick the fixed ratio want to display
# grid_num: default value = 20, it determine how smooth we would like the contour figure to be, higher grid number lead to more time to compute (approximation 20 leads to 1 min) 
#
# Output: none but gives a figure
def classification_model_plot(X_data,Y_data,scaler_x, scaler_y,model,sample_index,grid_num=20):
    X_data_inverse = scaler_x.inverse_transform(X_data)

    x1, x2, x3 = X_data_inverse[:, 0], X_data_inverse[:, 1], X_data_inverse[:,2]

    x1_data_min, x1_data_max = x1.min(), x1.max() # define min & max of feature 0
    x2_data_min, x2_data_max = x2.min(), x2.max() # define min & max of feature 0
    #x3_data_min, x3_data_max = x3.min() - 0.005, x3.max() + 0.005 # define min & max of feature 0

    #grid_num = 20
    plot_step_1 = (x1_data_max-x1_data_min)/grid_num # defining the meshgrid step size
    plot_step_2 = (x1_data_max-x1_data_min)/grid_num
    plot_step = min((plot_step_1,plot_step_2))

    X1_data_grid, X2_data_grid = np.meshgrid(np.arange(x1_data_min, x1_data_max, plot_step),
                                            np.arange(x2_data_min, x2_data_max, plot_step))

    len(np.arange(x1_data_min, x1_data_max, plot_step))
    len(np.arange(x2_data_min, x2_data_max, plot_step))

    X1_data_space = np.zeros((len(np.arange(x1_data_min, x1_data_max, plot_step)),len(np.arange(x2_data_min, x2_data_max, plot_step)),len(x3)))
    X2_data_space = np.zeros((len(np.arange(x1_data_min, x1_data_max, plot_step)),len(np.arange(x2_data_min, x2_data_max, plot_step)),len(x3)))
    X3_data_space = np.zeros((len(np.arange(x1_data_min, x1_data_max, plot_step)),len(np.arange(x2_data_min, x2_data_max, plot_step)),len(x3)))

    for i in range(len(np.arange(x1_data_min, x1_data_max, plot_step))):
        for j in range(len(np.arange(x2_data_min, x2_data_max, plot_step))):
            for k in range(len(x3)):
                X1_data_space[i,j,k] = X1_data_grid[j,i]
                X2_data_space[i,j,k] = X2_data_grid[j,i]
                X3_data_space[i,j,k] = x3[k]

    y_class_SVM_pred_disp = model.predict(scaler_x.transform(np.c_[X1_data_space.ravel(), X2_data_space.ravel(), X3_data_space.ravel()]))
    y_class_SVM_pred_disp_grid = y_class_SVM_pred_disp.reshape(X1_data_space.shape)

    temp_sample = sample_index
    temp_disp = scaler_y.inverse_transform(y_class_SVM_pred_disp_grid[:,:,temp_sample])

    temp_x1 = X1_data_space[:,:,temp_sample]
    temp_x2 = X2_data_space[:,:,temp_sample]
    temp_x3 = x3[temp_sample]

    plot_colors = 'ryb' # defining the 3 colors for each category
    n_classes = 3
    target_names = ['Not coilable','coilable','coilable but yield']

    fig2, ax2 = plt.subplots(tight_layout=True)
    plt.subplots_adjust(wspace=0.4, hspace=0.4)

    ax2.contourf(temp_x2, temp_x1, temp_disp, cmap=cm.RdYlBu, alpha=0.8)

    # Plot the training points
    ax2.scatter(X_data_inverse[temp_sample, 1], X_data_inverse[temp_sample, 0], c=plot_colors[Y_data[temp_sample]],
                    label=(Y_data[temp_sample],X_data_inverse[temp_sample, 2]), edgecolor='black', s=15)

    ax2.set_ylim(temp_x1.min(), temp_x1.max())
    ax2.set_xlim(temp_x2.min(), temp_x2.max())
    ax2.set_ylabel('ratio_d', fontsize=20)
    ax2.set_xlabel('ratio_pitch', fontsize=20)
    #ax2.set_xticks(())
    #ax2.set_yticks(())
    ax2.legend(loc='lower right', borderpad=0, handletextpad=0, fontsize=15)
    ax2.set_title('Support Vector Machine Classifier (SVC) with RBF kernel with ratio_top_diameter', fontsize=20)
    

In [10]:
# Define function to do the model training and predicting
# Input: X_train,X_test,X_data,Y_train,Y_test,Y_data,scaler_x, scaler_y,model
# X_train,X_test,X_data,Y_train,Y_test,Y_data,scaler_x, scaler_y,: scaled data set for classification
# model: any model for predict classification problem
#
# Output: ac_score,pr_score,re_score,f_one_score,con_matrix
# ac_score: accuracy_score with test sample
# pr_score: precision_score with test sample
# re_score: recall_score with test sample
# f_one_score: f1_score with test sample
# con_matrix: confusion_matrix with test sample 

def classification_model_train_and_predict(X_train,X_test,X_data,Y_train,Y_test,Y_data,scaler_x, scaler_y,model):
    model.fit(X_train,Y_train)
    Y_predict_for_test = model.predict(X_test)
    ac_score = accuracy_score(Y_test,Y_predict_for_test)
    pr_score = precision_score(Y_test,Y_predict_for_test,average='micro')
    re_score = recall_score(Y_test,Y_predict_for_test,average='micro')
    f_one_score = f1_score(Y_test,Y_predict_for_test,average='micro')
    con_matrix = confusion_matrix(Y_test,Y_predict_for_test)

    #classification_model_plot(X_data,Y_data,scaler_x, scaler_y,model,249,20)

    return ac_score,pr_score,re_score,f_one_score,con_matrix

In [7]:
# Define SVM model
svm_model = svm.SVC(kernel='rbf')

# Set up the parameter grid
parameter_grid = {'C': [0.75, 0.8, 0.85, 0.9, 0.95, 1, 1.05, 1.1, 1.15, 1.2, 1.25], 
                  'gamma': [0.75, 0.8, 0.85, 0.9, 0.95, 1, 1.05, 1.1, 1.15, 1.2, 1.25]}

# Configure GridSearchCV
grid_search = GridSearchCV(svm_model, parameter_grid, cv=5)

# Fit the grid search model
grid_search.fit(X_train_scale_class, Y_train_scale_class)

# Find the best parameters and score
best_parameters = grid_search.best_params_
best_score = grid_search.best_score_

# Create and fit the optimal model
optimal_model = svm.SVC(kernel='rbf', C=best_parameters['C'], gamma=best_parameters['gamma'])

ac_score,pr_score,re_score,f_one_score,con_matrix = classification_model_train_and_predict(X_train_scale_class, X_test_scale_class, X_scale_class, Y_train_scale_class, Y_test_scale_class, Y_scale_class,scaler_x_class, scaler_y_class,optimal_model)
print(ac_score)
print(pr_score)
print(re_score)
print(f_one_score)
print(con_matrix)

0.816
0.816
0.816
0.816
[[ 69   6   3]
 [  6  32  15]
 [  6  10 103]]


In [8]:
# Define Decision Tree model
dt_model = DecisionTreeClassifier()

# Set up the parameter grid
parameter_grid = {
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Configure GridSearchCV
grid_search = GridSearchCV(dt_model, parameter_grid, cv=5)

# Fit the grid search model
grid_search.fit(X_train_scale_class, Y_train_scale_class)

# Find the best parameters and score
best_parameters = grid_search.best_params_
best_score = grid_search.best_score_

# Create and fit the optimal model
optimal_model = DecisionTreeClassifier(**best_parameters)

ac_score,pr_score,re_score,f_one_score,con_matrix = classification_model_train_and_predict(X_train_scale_class, X_test_scale_class, X_scale_class, Y_train_scale_class, Y_test_scale_class, Y_scale_class,scaler_x_class, scaler_y_class,optimal_model)
print(ac_score)
print(pr_score)
print(re_score)
print(f_one_score)
print(con_matrix)

0.812
0.812
0.812
0.8119999999999999
[[ 66   7   5]
 [  5  35  13]
 [  5  12 102]]


In [9]:
# Define Random Forest model
rf_model = RandomForestClassifier()

# Set up the parameter grid
parameter_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Configure GridSearchCV
grid_search = GridSearchCV(rf_model, parameter_grid, cv=5)

# Fit the grid search model
grid_search.fit(X_train_scale_class, Y_train_scale_class)

# Find the best parameters and score
best_parameters = grid_search.best_params_
best_score = grid_search.best_score_

# Create and fit the optimal model with best parameters
optimal_model = RandomForestClassifier(**best_parameters)

ac_score,pr_score,re_score,f_one_score,con_matrix = classification_model_train_and_predict(X_train_scale_class, X_test_scale_class, X_scale_class, Y_train_scale_class, Y_test_scale_class, Y_scale_class,scaler_x_class, scaler_y_class,optimal_model)
print(ac_score)
print(pr_score)
print(re_score)
print(f_one_score)
print(con_matrix)


0.852
0.852
0.852
0.852
[[ 70   5   3]
 [  2  34  17]
 [  4   6 109]]


In [10]:
# Define Logistic Regression model
lr_model = LogisticRegression()

# Set up the parameter grid
parameter_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l2']
}

# Configure GridSearchCV
grid_search = GridSearchCV(lr_model, parameter_grid, cv=5)

# Fit the grid search model
grid_search.fit(X_train_scale_class, Y_train_scale_class)

# Find the best parameters and score
best_parameters = grid_search.best_params_
best_score = grid_search.best_score_

# Create and fit the optimal model with best parameters
optimal_model = LogisticRegression(**best_parameters)

ac_score,pr_score,re_score,f_one_score,con_matrix = classification_model_train_and_predict(X_train_scale_class, X_test_scale_class, X_scale_class, Y_train_scale_class, Y_test_scale_class, Y_scale_class,scaler_x_class, scaler_y_class,optimal_model)
print(ac_score)
print(pr_score)
print(re_score)
print(f_one_score)
print(con_matrix)

0.764
0.764
0.764
0.764
[[62  6 10]
 [ 8 31 14]
 [11 10 98]]


In [16]:
#kernel = 1.0 * RBF(1.0)  # RBF Kernel
kernel = Matern(length_scale=1.0, length_scale_bounds=(1e-2, 1e2),nu=2.5) # Matern Kernel

gpc = GaussianProcessClassifier(kernel=kernel, random_state=0)

ac_score,pr_score,re_score,f_one_score,con_matrix = classification_model_train_and_predict(X_train_scale_class, X_test_scale_class, X_scale_class, Y_train_scale_class, Y_test_scale_class, Y_scale_class,scaler_x_class, scaler_y_class,gpc)
print(ac_score)
print(pr_score)
print(re_score)
print(f_one_score)
print(con_matrix)

0.808
0.808
0.808
0.808
[[ 70   6   2]
 [  6  32  15]
 [  8  11 100]]


In [17]:
kernel = 1.0 * RBF(1.0)  # RBF Kernel
#kernel = Matern(length_scale=1.0, length_scale_bounds=(1e-2, 1e2),nu=2.5) # Matern Kernel

gpc = GaussianProcessClassifier(kernel=kernel, random_state=0)

ac_score,pr_score,re_score,f_one_score,con_matrix = classification_model_train_and_predict(X_train_scale_class, X_test_scale_class, X_scale_class, Y_train_scale_class, Y_test_scale_class, Y_scale_class,scaler_x_class, scaler_y_class,gpc)
print(ac_score)
print(pr_score)
print(re_score)
print(f_one_score)
print(con_matrix)

0.804
0.804
0.804
0.804
[[70  6  2]
 [ 6 32 15]
 [ 8 12 99]]


In [7]:
# Define function to display classification when fixing "ratio_top_diameter"
# Input: X_data,Y_data,scaler_x, scaler_y,model,sample_index,grid_num=20
# X_data, Y_data, scaler_x: scaled data set for classification
# model: any model for predict classification problem
# sample_index: index of "ratio_top_diameter" pick the fixed ratio want to display
# grid_num: default value = 20, it determine how smooth we would like the contour figure to be, higher grid number lead to more time to compute (approximation 20 leads to 1 min) 
#
# Output: none but gives a figure
def regression_model_plot(X_data,Y_data,scaler_x, scaler_y,model,sample_index,grid_num=20,problem_sigma = True):
    X_data_inverse = scaler_x.inverse_transform(X_data)
    Y_data_inverse = scaler_y.inverse_transform(Y_data)
    x1, x2, x3 = X_data_inverse[:, 0], X_data_inverse[:, 1], X_data_inverse[:,2]

    x1_data_min, x1_data_max = x1.min(), x1.max() # define min & max of feature 0
    x2_data_min, x2_data_max = x2.min(), x2.max() # define min & max of feature 0
    #x3_data_min, x3_data_max = x3.min() - 0.005, x3.max() + 0.005 # define min & max of feature 0

    #grid_num = 20
    plot_step_1 = (x1_data_max-x1_data_min)/grid_num # defining the meshgrid step size
    plot_step_2 = (x1_data_max-x1_data_min)/grid_num
    plot_step = min((plot_step_1,plot_step_2))

    X1_data_grid, X2_data_grid = np.meshgrid(np.arange(x1_data_min, x1_data_max, plot_step),
                                            np.arange(x2_data_min, x2_data_max, plot_step))

    len(np.arange(x1_data_min, x1_data_max, plot_step))
    len(np.arange(x2_data_min, x2_data_max, plot_step))

    X1_data_space = np.zeros((len(np.arange(x1_data_min, x1_data_max, plot_step)),len(np.arange(x2_data_min, x2_data_max, plot_step)),len(x3)))
    X2_data_space = np.zeros((len(np.arange(x1_data_min, x1_data_max, plot_step)),len(np.arange(x2_data_min, x2_data_max, plot_step)),len(x3)))
    X3_data_space = np.zeros((len(np.arange(x1_data_min, x1_data_max, plot_step)),len(np.arange(x2_data_min, x2_data_max, plot_step)),len(x3)))

    for i in range(len(np.arange(x1_data_min, x1_data_max, plot_step))):
        for j in range(len(np.arange(x2_data_min, x2_data_max, plot_step))):
            for k in range(len(x3)):
                X1_data_space[i,j,k] = X1_data_grid[j,i]
                X2_data_space[i,j,k] = X2_data_grid[j,i]
                X3_data_space[i,j,k] = x3[k]

    if problem_sigma == True:
        y_class_SVM_pred_disp = model.predict(scaler_x.transform(np.c_[X1_data_space.ravel(), X2_data_space.ravel(), X3_data_space.ravel()]))
        y_class_SVM_pred_disp_inverse = scaler_y.inverse_transform(y_class_SVM_pred_disp)

        y_class_SVM_pred_disp_grid = y_class_SVM_pred_disp_inverse[:,0].reshape(X1_data_space.shape)

        temp_sample = sample_index
        temp_disp = y_class_SVM_pred_disp_grid[:,:,temp_sample]

        temp_x1 = X1_data_space[:,:,temp_sample]
        temp_x2 = X2_data_space[:,:,temp_sample]
        temp_x3 = x3[temp_sample]

        #plot_colors = 'ryb' # defining the 3 colors for each category
        #n_classes = 3
        #target_names = ['Not coilable','coilable','coilable but yield']

        fig2, ax3 = plt.subplots(tight_layout=True)
        plt.subplots_adjust(wspace=0.4, hspace=0.4)

        
        ax3.contourf(temp_x2, temp_x1, temp_disp, cmap=cm.RdYlBu, alpha=0.8)
        # Plot the training points
        ax3.scatter(X_data_inverse[temp_sample, 1], X_data_inverse[temp_sample, 0], c=Y_data_inverse[temp_sample,0],
                        label=(Y_data[temp_sample,0],X_data_inverse[temp_sample, 2]), edgecolor='black', s=15)
    else:
        y_class_SVM_pred_disp = model.predict(scaler_x.transform(np.c_[X1_data_space.ravel(), X2_data_space.ravel(), X3_data_space.ravel()]))
        y_class_SVM_pred_disp_inverse = scaler_y.inverse_transform(y_class_SVM_pred_disp)

        y_class_SVM_pred_disp_grid = y_class_SVM_pred_disp_inverse[:,1].reshape(X1_data_space.shape)

        temp_sample = sample_index
        temp_disp = y_class_SVM_pred_disp_grid[:,:,temp_sample]

        temp_x1 = X1_data_space[:,:,temp_sample]
        temp_x2 = X2_data_space[:,:,temp_sample]
        temp_x3 = x3[temp_sample]

        fig2, ax3 = plt.subplots(tight_layout=True)
        plt.subplots_adjust(wspace=0.4, hspace=0.4)

        
        ax3.contourf(temp_x2, temp_x1, temp_disp, cmap=cm.RdYlBu, alpha=0.8)
        # Plot the training points
        ax3.scatter(X_data_inverse[temp_sample, 1], X_data_inverse[temp_sample, 0], c=Y_data_inverse[temp_sample,1],
                        label=(Y_data[temp_sample,1],X_data_inverse[temp_sample, 2]), edgecolor='black', s=15)

    ax3.set_ylim(temp_x1.min(), temp_x1.max())
    ax3.set_xlim(temp_x2.min(), temp_x2.max())
    ax3.set_ylabel('ratio_d', fontsize=20)
    ax3.set_xlabel('ratio_pitch', fontsize=20)
    #ax2.set_xticks(())
    #ax2.set_yticks(())
    ax3.legend(loc='lower right', borderpad=0, handletextpad=0, fontsize=15)
    ax3.set_title('Support Vector Machine Classifier (SVC) with RBF kernel with ratio_top_diameter', fontsize=20)
    fig2.colorbar(cm.ScalarMappable(norm=None, cmap=cm.RdYlBu), ax=ax3)
    

In [8]:
def regression_model_train_and_predict(X_train,X_test,X_data,Y_train,Y_test,Y_data,scaler_x,scaler_y,model):
    model.fit(X_train,Y_train)
    Y_predict_for_test = model.predict(X_test)
    r2 = r2_score(Y_test,Y_predict_for_test)
    mse = mean_squared_error(Y_test,Y_predict_for_test)

    #regression_model_plot(X_data,Y_data,scaler_x,scaler_y,model,5,10,True)
    # this function is not finished, and we can just check r2 score and mse for comparing our results

    return r2, mse

In [9]:
kernel = Matern(length_scale=1.0, length_scale_bounds=(1e-2, 1e2),nu=2.5)
gpr_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-3, optimizer='fmin_l_bfgs_b', n_restarts_optimizer=20)

r2, mse = regression_model_train_and_predict(X_train_scale_regression, X_test_scale_regression, X_scale_regression, Y_train_scale_regression, Y_test_scale_regression, Y_scale_regression,scaler_x_regression, scaler_y_regression,gpr_model)

print(r2)
print(mse)

0.9993278047777596
0.0006239489505658043


In [10]:
degrees = [3, 4, 5, 6, 7]

# Prepare to store results
results = []

for degree in degrees:
    # Create a pipeline that first creates polynomial features and then applies Ridge regression
    model = make_pipeline(PolynomialFeatures(degree), RidgeCV())

    # Perform cross-validation and store the mean score
    scores = cross_val_score(model, X_scale_regression, Y_scale_regression, cv=5, scoring='neg_mean_squared_error')
    mean_score = np.mean(scores)
    results.append((degree, mean_score))

degrees, scores = zip(*results)

best_degree = max(results, key=lambda item: item[1])[0]
model = make_pipeline(PolynomialFeatures(best_degree), RidgeCV())

r2, mse = regression_model_train_and_predict(X_train_scale_regression, X_test_scale_regression, X_scale_regression, Y_train_scale_regression, Y_test_scale_regression, Y_scale_regression,scaler_x_regression, scaler_y_regression,model)

print(r2)
print(mse)
print(best_degree)

0.9993323050439886
0.0006212911982154898
5


In [7]:
# define a function to do the preprocessing
# Input: input_raw_data, output_raw_data, problem_class
# input_raw_data: pandas dataframe for input
# output_raw_data: pandas dataframe for output
# problem_class: (booler) to describe this preprocess is for classification problem or not
#                this will lead to different scaler for Y data.
#
# Output: X_train_scale, X_test_scale, X_scale, Y_train_scale, Y_test_scale, Y_scale, scaler_x, scaler_y
# scaler_x: (scaler) used to do inverse_transfer after prediction
# scaler_y: (scaler) used to do inverse_transfer after prediction

def preprocess_7d(input_raw_data,output_raw_data,problem_class):
    if problem_class == False:
        raw_data = pd.concat([input_raw_data, output_raw_data], axis=1)

        # look at the # of missing points in the first ten columns
        raw_data_drop_nan = raw_data.dropna()

        X_7d = raw_data_drop_nan.loc[:, ['ratio_area','ratio_Ixx','ratio_Iyy','ratio_J','ratio_pitch','ratio_top_diameter','ratio_shear_modulus'] ].values
        Y_7d = raw_data_drop_nan.loc[:, ['sigma_crit','energy'] ].values
        
        testset_ratio = 0.25
        SEED = 123
        X_train, X_test, Y_train, Y_test = train_test_split(X_7d,
                                            Y_7d, test_size=testset_ratio,
                                            random_state=SEED)
        
        scaler_x = StandardScaler()
        scaler_x.fit(X_train)
        X_train_scale=scaler_x.transform(X_train)
        X_test_scale=scaler_x.transform(X_test)
        X_scale=scaler_x.transform(X_7d)
        
        scaler_y = FunctionTransformer(np.log1p, validate=True)
        #scaler_y = StandardScaler()
        scaler_y.fit(Y_train)
        Y_train_scale = scaler_y.transform(Y_train)
        Y_test_scale = scaler_y.transform(Y_test)
        Y_scale = scaler_y.transform(Y_7d)
    else:
        X_7d = input_raw_data.loc[:, ['ratio_area','ratio_Ixx','ratio_Iyy','ratio_J','ratio_pitch','ratio_top_diameter','ratio_shear_modulus'] ].values
        Y_7d = output_raw_data.loc[:, 'coilable'].values
        testset_ratio = 0.25
        SEED = 123
        X_train, X_test, Y_train, Y_test = train_test_split(X_7d,
                                            Y_7d, test_size=testset_ratio,
                                            random_state=SEED)

        scaler_x = StandardScaler()
        scaler_x.fit(X_train)
        X_train_scale=scaler_x.transform(X_train)
        X_test_scale=scaler_x.transform(X_test)
        X_scale=scaler_x.transform(X_7d)

        scaler_y = FunctionTransformer() # FunctionTransformer without input will give a Identity scaler
        Y_train_scale = scaler_y.transform(Y_train)
        Y_test_scale = scaler_y.transform(Y_test)
        Y_scale = scaler_y.transform(Y_7d)

    return X_train_scale, X_test_scale, X_scale, Y_train_scale, Y_test_scale, Y_scale, scaler_x, scaler_y

In [12]:
X_train_scale_class_7d, X_test_scale_class_7d, X_scale_class_7d, Y_train_scale_class_7d, Y_test_scale_class_7d, Y_scale_class_7d, scaler_x_class_7d, scaler_y_class_7d = preprocess_7d(df_in_7d,df_out_7d,True)
X_train_scale_regression_7d, X_test_scale_regression_7d, X_scale_regression_7d, Y_train_scale_regression_7d, Y_test_scale_regression_7d, Y_scale_regression_7d, scaler_x_regression_7d, scaler_y_regression_7d = preprocess_7d(df_in_7d,df_out_7d,False)

In [18]:
SVM_model = svm.SVC(kernel='rbf',gamma=0.7,C=1.0)

ac_score,pr_score,re_score,f_one_score,con_matrix = classification_model_train_and_predict(X_train_scale_class_7d, X_test_scale_class_7d, X_scale_class_7d, Y_train_scale_class_7d, Y_test_scale_class_7d, Y_scale_class_7d,scaler_x_class_7d, scaler_y_class_7d,SVM_model)
print(ac_score)
print(pr_score)
print(re_score)
print(f_one_score)
print(con_matrix)

0.95832
0.95832
0.95832
0.95832
[[4027  322]
 [ 199 7952]]


In [16]:
#kernel = 1.0 * RBF(1.0)  # RBF Kernel
kernel = Matern(length_scale=1.0, length_scale_bounds=(1e-2, 1e2),nu=2.5) # Matern Kernel

gpc = GaussianProcessClassifier(kernel=kernel, random_state=0)

ac_score,pr_score,re_score,f_one_score,con_matrix = classification_model_train_and_predict(X_train_scale_class_7d, X_test_scale_class_7d, X_scale_class_7d, Y_train_scale_class_7d, Y_test_scale_class_7d, Y_scale_class_7d,scaler_x_class_7d, scaler_y_class_7d,gpc)
print(ac_score)
print(pr_score)
print(re_score)
print(f_one_score)
print(con_matrix)

: 

In [14]:
# Define Random Forest model
rf_model = RandomForestClassifier()

# Set up the parameter grid
parameter_grid = {
    'n_estimators': [10, 100, 200],
    'max_depth': [None, 10, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Configure GridSearchCV
grid_search = GridSearchCV(rf_model, parameter_grid, cv=5)

# Fit the grid search model
grid_search.fit(X_train_scale_class_7d, Y_train_scale_class_7d)

# Find the best parameters and score
best_parameters = grid_search.best_params_
best_score = grid_search.best_score_

# Create and fit the optimal model with best parameters
optimal_model = RandomForestClassifier(**best_parameters)

ac_score,pr_score,re_score,f_one_score,con_matrix = classification_model_train_and_predict(X_train_scale_class_7d, X_test_scale_class_7d, X_scale_class_7d, Y_train_scale_class_7d, Y_test_scale_class_7d, Y_scale_class_7d,scaler_x_class_7d, scaler_y_class_7d,optimal_model)
print(ac_score)
print(pr_score)
print(re_score)
print(f_one_score)
print(con_matrix)

0.95216
0.95216
0.95216
0.95216
[[3966  383]
 [ 215 7936]]


In [15]:
# Define Decision Tree model
dt_model = DecisionTreeClassifier()

# Set up the parameter grid
parameter_grid = {
    'max_depth': [None, 10, 30, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Configure GridSearchCV
grid_search = GridSearchCV(dt_model, parameter_grid, cv=5)

# Fit the grid search model
grid_search.fit(X_train_scale_class_7d, Y_train_scale_class_7d)

# Find the best parameters and score
best_parameters = grid_search.best_params_
best_score = grid_search.best_score_

# Create and fit the optimal model
optimal_model = DecisionTreeClassifier(**best_parameters)

ac_score,pr_score,re_score,f_one_score,con_matrix = classification_model_train_and_predict(X_train_scale_class_7d, X_test_scale_class_7d, X_scale_class_7d, Y_train_scale_class_7d, Y_test_scale_class_7d, Y_scale_class_7d,scaler_x_class_7d, scaler_y_class_7d,optimal_model)
print(ac_score)
print(pr_score)
print(re_score)
print(f_one_score)
print(con_matrix)

0.91048
0.91048
0.91048
0.91048
[[3822  527]
 [ 592 7559]]


In [17]:
kernel = Matern(length_scale=1.0, length_scale_bounds=(1e-2, 1e2),nu=2.5)
gpr_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-3, optimizer='fmin_l_bfgs_b', n_restarts_optimizer=20)

r2, mse = regression_model_train_and_predict(X_train_scale_regression_7d, X_test_scale_regression_7d, X_scale_regression_7d, Y_train_scale_regression_7d, Y_test_scale_regression_7d, Y_scale_regression_7d,scaler_x_regression_7d, scaler_y_regression_7d,gpr_model)

In [18]:
print(r2)
print(mse)

0.964030829707237
0.03206111673290605


In [24]:
degrees = [3, 4, 5, 6, 7]

# Prepare to store results
results = []

for degree in degrees:
    # Create a pipeline that first creates polynomial features and then applies Ridge regression
    model = make_pipeline(PolynomialFeatures(degree), RidgeCV())

    # Perform cross-validation and store the mean score
    scores = cross_val_score(model, X_scale_regression_7d, Y_scale_regression_7d, cv=5, scoring='neg_mean_squared_error')
    mean_score = np.mean(scores)
    results.append((degree, mean_score))

degrees, scores = zip(*results)

best_degree = max(results, key=lambda item: item[1])[0]
model = make_pipeline(PolynomialFeatures(best_degree), RidgeCV())

r2, mse = regression_model_train_and_predict(X_train_scale_regression_7d, X_test_scale_regression_7d, X_scale_regression_7d, Y_train_scale_regression_7d, Y_test_scale_regression_7d, Y_scale_regression_7d,scaler_x_regression_7d, scaler_y_regression_7d,model)

print(r2)
print(mse)
print(best_degree)

0.9784282756778737
0.017931286373653585
7
